# Retrieve Daily MODIS NDVI data from Google Earth Engine from every WFIGS 2022 fire

WFIGS 2022 geojson data retrieved from https://www.nwcg.gov/publications/pms936/nifs/public-distribution

## Read WFIGS 2022 file and describe the data set

Result: 

RangeIndex: 5320 entries, 0 to 5319

Columns: 109 entries, OBJECTID to geometry

dtypes: datetime64[ns, UTC](14), float64(26), geometry(1), int64(3), object(65)

memory usage: 4.4+ MB

In [3]:
# Import libraries
import geopandas as gpd
import timeit
import sys
import json
import ee

# Authenticate to google earth engine
service_account = 'account'
key_path = 'path'
credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials)

In [4]:
# Read in the dataset from geojson file to geopandas dataframe
gdf = gpd.read_file('WFIGS_-_2022_Wildland_Fire_Perimeters_to_Date.geojson')
gdf.head()

,OBJECTID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_Acres_AutoCalc,poly_GlobalID,...,irwin_ArchivedOn,irwin_ModifiedOnDateTime_dt,irwin_CreatedOnDateTime_dt,GlobalID,irwin_IsCpxChild,irwin_CpxName,irwin_CpxID,SHAPE_Length,SHAPE_Area,geometry
0,14680,Muck Farm,Wildfire Final Fire Perimeter,Phone/Tablet,2.990264,2022-06-14 16:18:45+00:00,2022-06-14 16:18:45+00:00,2022-06-14T16:18:45+00:00,2.962293,{FE959070-6FAC-4A71-8EA0-70EBAE10E6DF},...,None,2022-06-14 16:09:58+00:00,2022-01-04 23:22:37+00:00,{35A7B8E8-6143-4B2A-AE86-39E23AAF79A0},0.0,None,None,0.004723,-1.111475e-06,"MULTIPOLYGON (((-81.90191 29.13707, -81.90194 ..."
1,14683,Muck Farm 3,Wildfire Final Fire Perimeter,Hand Sketch,0.009985,2022-06-14 16:05:11+00:00,2022-06-14 16:05:11+00:00,2022-06-14T16:05:11+00:00,0.009985,{0C87AA07-CE71-49E8-93AA-BC91E1673F1A},...,None,2022-04-19 19:49:28+00:00,2022-01-05 16:23:59+00:00,{1D31F0F5-E2B6-4735-8812-C9992E1A469E},0.0,None,None,0.000230,-3.745838e-09,"MULTIPOLYGON (((-81.90985 29.11920, -81.90981 ..."
2,14698,Day,Wildfire Daily Fire Perimeter,Image Interpretation,257.454151,2022-01-05 23:28:22+00:00,2022-10-21 14:55:45+00:00,None,257.453233,{137A0F2D-41CF-4B75-96A4-F9F0E4F670BE},...,None,2022-01-10 16:10:56+00:00,2022-01-01 18:01:19+00:00,{29180A93-0245-44BD-A392-13592BD678AB},0.0,None,None,0.068949,-9.838932e-05,"MULTIPOLYGON (((-92.92398 31.00099, -92.92393 ..."
3,14700,Pelt,Wildfire Daily Fire Perimeter,Image Interpretation,NaN,2022-01-19 22:56:18+00:00,2022-01-19 22:56:18+00:00,None,113.023344,{A8C4215D-4BA9-485C-BA50-A4701421311C},...,None,2022-01-19 22:56:16+00:00,2022-01-01 16:50:50+00:00,{524E1D0F-13EE-495B-AE84-0DE5C87D4844},0.0,None,None,0.062518,-4.320592e-05,"MULTIPOLYGON (((-92.95555 31.02011, -92.95555 ..."
4,14805,Beaver Pond,Wildfire Daily Fire Perimeter,GPS-Walked/Driven,432.102100,2022-01-07 21:54:11+00:00,2022-10-21 14:55:48+00:00,2022-01-07T12:57:00+00:00,432.100593,{DB72D8E1-17A9-4F13-AEB1-375BBC732716},...,None,2022-01-10 14:18:52+00:00,2022-01-07 05:02:31+00:00,{F8C884FE-0A3B-4E13-8DEF-433DF489E368},0.0,None,None,0.076152,-1.660243e-04,"MULTIPOLYGON (((-94.69934 31.51343, -94.69934 ..."


Print the number of rows and columns in the data set

In [5]:
print(gdf.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5320 entries, 0 to 5319
Columns: 109 entries, OBJECTID to geometry
dtypes: datetime64[ns, UTC](14), float64(26), geometry(1), int64(3), object(65)
memory usage: 4.4+ MB
None


## Data cleaning: check for NaN, None, and duplicate records. 

Results: 

- Keep the NaN and None as they have geometries that may still be valuable. 
- Drop the duplicates and keep the record with largest area
- nan
- None
- Drop duplicates and keep largest area
- Subset by only rows where poly_FeatureCategory = 'Wildfire Final Fire Perimeter'

Many columns have nan, but they all have a geometry, so we will not drop any columns based on nan

In [6]:
print(f'Columns with nan records: {gdf.columns[gdf.isna().any()].tolist()}')

Columns with nan records: ['poly_IncidentName', 'poly_MapMethod', 'poly_GISAcres', 'poly_PolygonDateTime', 'irwin_ABCDMisc', 'irwin_CalculatedAcres', 'irwin_ContainmentDateTime', 'irwin_ControlDateTime', 'irwin_DailyAcres', 'irwin_DiscoveryAcres', 'irwin_EstimatedCostToDate', 'irwin_FFReportApprovedByTitle', 'irwin_FFReportApprovedByUnit', 'irwin_FFReportApprovedDate', 'irwin_FireBehaviorGeneral', 'irwin_FireBehaviorGeneral1', 'irwin_FireBehaviorGeneral2', 'irwin_FireBehaviorGeneral3', 'irwin_FireCause', 'irwin_FireCauseGeneral', 'irwin_FireCauseSpecific', 'irwin_FireCode', 'irwin_FireDepartmentID', 'irwin_FireMgmtComplexity', 'irwin_FireOutDateTime', 'irwin_FSConfinePercent', 'irwin_FSFullSuppPercent', 'irwin_FSMonitorPercent', 'irwin_FSPointZonePercent', 'irwin_FSJobCode', 'irwin_FSOverrideCode', 'irwin_ICS209ReportDateTime', 'irwin_ICS209RForTimePeriodFrom', 'irwin_ICS209RForTimePeriodTo', 'irwin_ICS209ReportStatus', 'irwin_IncidentManagementOrg', 'irwin_IncidentShortDescription', '

Count of Nulls (displayed as None) in poly_IncidentName: 20

Result: they all have geometries and their states or named locations could be found by doing a reverse lookup to see which state they belong to. We will keep these in the data set as their locations may still be a valid source of data for wildfire NDVI information.

In [7]:
print(f'Count of Nulls in poly_IncidentName: {gdf.poly_IncidentName.isnull().sum()}')
gdf[gdf['poly_IncidentName'].isnull()]

Count of Nulls in poly_IncidentName: 20


,OBJECTID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_Acres_AutoCalc,poly_GlobalID,...,irwin_ArchivedOn,irwin_ModifiedOnDateTime_dt,irwin_CreatedOnDateTime_dt,GlobalID,irwin_IsCpxChild,irwin_CpxName,irwin_CpxID,SHAPE_Length,SHAPE_Area,geometry
40,15045,None,Wildfire Daily Fire Perimeter,Mixed Methods,NaN,2022-03-14 18:02:08+00:00,2022-03-14 18:02:08+00:00,None,80.400076,{40143BFD-72C5-449C-81EC-53C3BC751008},...,None,2022-03-14 18:02:05+00:00,2022-01-31 23:20:40+00:00,{B31C46C1-9F6D-438A-A437-0BA818CC5600},0.0,None,None,0.024427,-3.284887e-05,"MULTIPOLYGON (((-91.27068 36.78735, -91.27066 ..."
44,15052,None,Wildfire Daily Fire Perimeter,Mixed Methods,NaN,2022-02-08 14:16:15+00:00,2022-02-08 14:16:15+00:00,None,8.253283,{AEDEE0A3-8395-47E4-966B-BF02C4598020},...,None,2022-02-08 14:16:12+00:00,2022-01-28 16:08:06+00:00,{550A2C33-212D-432D-BA42-20E16E615DFC},0.0,None,None,0.007650,-3.299085e-06,"MULTIPOLYGON (((-93.32620 35.02845, -93.32618 ..."
98,15129,None,Wildfire Daily Fire Perimeter,GPS-Walked,NaN,2022-02-08 21:59:41+00:00,2022-02-08 21:59:41+00:00,None,0.744902,{DB4FD097-AA60-4F5E-AE91-7855B51BFE29},...,None,2022-02-08 21:59:40+00:00,2022-01-31 15:31:36+00:00,{4E9EF7E2-8A92-4D33-84DF-8F5913E91234},0.0,None,None,0.002905,-3.369557e-07,"MULTIPOLYGON (((-103.22322 43.76714, -103.2232..."
115,15157,None,Wildfire Daily Fire Perimeter,Mixed Methods,NaN,2022-02-16 18:55:35+00:00,2022-02-16 18:55:35+00:00,None,1.478692,{B2068A42-E5E4-4652-A2F0-34EBD83BFEB4},...,None,2022-02-16 18:55:33+00:00,2022-02-12 00:27:32+00:00,{7782D5ED-66E6-437E-8AE4-CC6569B8735A},0.0,None,None,0.003756,-6.140795e-07,"MULTIPOLYGON (((-83.50193 38.03339, -83.50193 ..."
2290,17757,None,Wildfire Final Fire Perimeter,GPS-Flight,1.250000,2022-08-05 21:13:53+00:00,2022-08-05 21:13:53+00:00,2022-08-05T21:13:53+00:00,1.459844,{32D97A7C-C02D-4DB8-AE08-382F62FDD1BA},...,None,2022-07-05 12:15:25+00:00,2022-07-05 12:15:25+00:00,{26E31C35-3131-429A-BAA3-FF76C9C793C7},0.0,None,None,0.003871,-5.505455e-07,"MULTIPOLYGON (((-81.79312 29.66607, -81.79259 ..."
3077,18564,None,Wildfire Final Fire Perimeter,Mixed Methods,NaN,2022-08-17 00:04:59+00:00,2022-08-17 00:04:59+00:00,2022-08-17T00:04:57+00:00,73.805274,{DF0E22F3-C23E-4D2F-86E4-0D33BCF70D4B},...,None,2022-08-23 22:42:32+00:00,2022-08-01 08:22:08+00:00,{CD294307-B04C-4E4D-B317-2FC5B96C652F},0.0,None,None,0.044675,-3.496818e-05,"MULTIPOLYGON (((-116.97455 46.44282, -116.9744..."
3239,18730,None,Wildfire Daily Fire Perimeter,Mixed Methods,NaN,2022-06-21 17:55:50+00:00,2022-08-05 21:46:42+00:00,None,0.069455,{6A70D4A0-B5C6-4F3F-9894-4F8ED554515A},...,None,2022-06-25 16:09:54+00:00,2022-06-20 17:28:31+00:00,{5A7E9BBD-19E4-421B-95F1-B12956CF3F51},0.0,None,None,0.000898,-3.134585e-08,"MULTIPOLYGON (((-85.90111 43.62706, -85.90108 ..."
3271,18762,None,Wildfire Final Fire Perimeter,GPS-Walked,NaN,2022-12-14 16:13:55+00:00,2022-12-14 16:13:55+00:00,2022-12-14T16:13:55+00:00,3.233381,{B9E17A3F-8C0E-41AA-AF0D-29EE96A9EAC3},...,None,2022-08-04 13:54:33+00:00,2022-08-04 13:54:28+00:00,{2ED527BF-FC48-4357-BE84-2AD0B62F0C02},0.0,None,None,0.005324,-1.427829e-06,"MULTIPOLYGON (((-71.74266 42.26682, -71.74263 ..."
3366,18858,None,Wildfire Daily Fire Perimeter,Mixed Methods,NaN,2022-08-01 21:29:21+00:00,2022-08-08 16:40:13+00:00,None,57.784959,{FBBB4CF4-FE55-494F-BD19-B290B7616E7F},...,None,2022-09-12 16:39:12+00:00,2022-08-01 21:41:05+00:00,{9300EB0A-FA29-4468-8211-6A713D5EC9B4},0.0,None,None,0.030924,-2.622128e-05,"MULTIPOLYGON (((-112.26940 43.95356, -112.2693..."
3583,19083,None,Wildfire Daily Fire Perimeter,Mixed Methods,NaN,2022-08-10 23:02:41+00:00,2022-08-15 15:56:09+00:00,None,343.536770,{1AFD07C6-56B0-4E9E-9631-684E9D21C50C},...,None,2022-09-12 18:13:28+00:00,2022-08-10 13:03:46+00:00,{FB859EB2-0CB9-447C-8906-151FED661BEB},0.0,None,None,0.071141,-1.530482e-04,"MULTIPOLYGON (((-112.53867 42.82411, -112.5386..."


Drop duplicate values of poly_IncidentName and keep the largest by SHAPE_Area

In [8]:
# identify duplicates
print(f"Number of duplicate records: {gdf[gdf.duplicated(['poly_IncidentName'], keep=False)].sort_values(by='poly_IncidentName').shape[0]}")
# gdf[gdf.duplicated(['poly_IncidentName'], keep=False)].sort_values(by='poly_IncidentName')

Number of duplicate records: 526


In [9]:
# drop duplicate values in the gdf dataframe
gdf = gdf.sort_values('SHAPE_Area', ascending=False).drop_duplicates('poly_IncidentName').sort_index()

# test to make sure the duplicate values were dropped
print(f"Number of duplicate records: {gdf[gdf.duplicated(['poly_IncidentName'], keep=False)].sort_values(by='poly_IncidentName').shape[0]}")

Number of duplicate records: 0


#### Subset by only rows where poly_FeatureCategory = 'Wildfire Final Fire Perimeter'

In [85]:
gdf = gdf[gdf['poly_FeatureCategory']=='Wildfire Final Fire Perimeter']

#### Subset by Months of June, July, and August

Frequency of observations by month

In [ ]:
import datetime as dt
gdf['poly_CreateDate'].groupby([gdf.poly_CreateDate.dt.year, gdf.poly_CreateDate.dt.month]).agg('count')

Filter to Months 6,7,8

In [94]:
# create new column for month
gdf['Month'] = gdf.poly_CreateDate.dt.month

# subset by Months 6,7,8
gdf = gdf[gdf.Month.isin([6,7,8])]

/Users/richardpeterson/opt/miniconda3/envs/pygis/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## Retrieve Daily MODIS NDVI readings from Google Earth Engine

Each record's geometry column will be passed to a function that retrieves the NDVI data from Google Earth Engine and writes the data to a file with format 'objectid_poly_IncidentName_dateretrieved'

In [10]:
gdf.head(10)

,OBJECTID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_Acres_AutoCalc,poly_GlobalID,...,irwin_ArchivedOn,irwin_ModifiedOnDateTime_dt,irwin_CreatedOnDateTime_dt,GlobalID,irwin_IsCpxChild,irwin_CpxName,irwin_CpxID,SHAPE_Length,SHAPE_Area,geometry
0,14680,Muck Farm,Wildfire Final Fire Perimeter,Phone/Tablet,2.990264,2022-06-14 16:18:45+00:00,2022-06-14 16:18:45+00:00,2022-06-14T16:18:45+00:00,2.962293,{FE959070-6FAC-4A71-8EA0-70EBAE10E6DF},...,None,2022-06-14 16:09:58+00:00,2022-01-04 23:22:37+00:00,{35A7B8E8-6143-4B2A-AE86-39E23AAF79A0},0.0,None,None,0.004723,-1.111475e-06,"MULTIPOLYGON (((-81.90191 29.13707, -81.90194 ..."
1,14683,Muck Farm 3,Wildfire Final Fire Perimeter,Hand Sketch,0.009985,2022-06-14 16:05:11+00:00,2022-06-14 16:05:11+00:00,2022-06-14T16:05:11+00:00,0.009985,{0C87AA07-CE71-49E8-93AA-BC91E1673F1A},...,None,2022-04-19 19:49:28+00:00,2022-01-05 16:23:59+00:00,{1D31F0F5-E2B6-4735-8812-C9992E1A469E},0.0,None,None,0.000230,-3.745838e-09,"MULTIPOLYGON (((-81.90985 29.11920, -81.90981 ..."
2,14698,Day,Wildfire Daily Fire Perimeter,Image Interpretation,257.454151,2022-01-05 23:28:22+00:00,2022-10-21 14:55:45+00:00,None,257.453233,{137A0F2D-41CF-4B75-96A4-F9F0E4F670BE},...,None,2022-01-10 16:10:56+00:00,2022-01-01 18:01:19+00:00,{29180A93-0245-44BD-A392-13592BD678AB},0.0,None,None,0.068949,-9.838932e-05,"MULTIPOLYGON (((-92.92398 31.00099, -92.92393 ..."
3,14700,Pelt,Wildfire Daily Fire Perimeter,Image Interpretation,NaN,2022-01-19 22:56:18+00:00,2022-01-19 22:56:18+00:00,None,113.023344,{A8C4215D-4BA9-485C-BA50-A4701421311C},...,None,2022-01-19 22:56:16+00:00,2022-01-01 16:50:50+00:00,{524E1D0F-13EE-495B-AE84-0DE5C87D4844},0.0,None,None,0.062518,-4.320592e-05,"MULTIPOLYGON (((-92.95555 31.02011, -92.95555 ..."
5,14814,Welding,Wildfire Daily Fire Perimeter,Hand Sketch,0.310674,2022-01-08 13:17:25+00:00,2022-10-21 14:55:48+00:00,None,0.310672,{1A02C437-4C65-4D0D-9A21-9B3248D4CB94},...,None,2022-01-10 20:53:04+00:00,2022-01-06 23:53:46+00:00,{BA1705FB-06AE-4980-B773-BA4DD521A77B},0.0,None,None,0.001512,-1.182842e-07,"MULTIPOLYGON (((-95.38490 30.62515, -95.38489 ..."
6,14859,Davis Street,Wildfire Daily Fire Perimeter,Hand Sketch,NaN,2022-01-14 15:35:55+00:00,2022-01-14 15:35:55+00:00,None,82.659351,{46195D43-1EA0-4C2E-A7C6-4637EF8BBF31},...,None,2022-01-14 15:35:53+00:00,2022-01-12 22:06:12+00:00,{5E6B01C3-BE97-4173-BB96-2469B88B8E56},0.0,None,None,0.028209,-3.341659e-05,"MULTIPOLYGON (((-101.61327 35.95191, -101.6131..."
7,14860,Sunrise,Wildfire Daily Fire Perimeter,Hand Sketch,NaN,2022-01-14 15:24:44+00:00,2022-01-14 15:24:44+00:00,None,66.242552,{796B79C4-D992-4E8B-B3B3-A12BED60929F},...,None,2022-01-14 15:24:43+00:00,2022-01-13 15:45:45+00:00,{FD5987D9-19F2-4595-9488-E1C2CFE7A48F},0.0,None,None,0.026809,-2.677976e-05,"MULTIPOLYGON (((-101.61321 35.95208, -101.6131..."
8,14873,Spring Creek,Wildfire Daily Fire Perimeter,Hand Sketch,NaN,2022-01-14 17:07:17+00:00,2022-01-14 17:07:17+00:00,None,20.247947,{AF7D191F-A32E-4E9C-A00D-6CFEFD42B25D},...,None,2022-01-14 17:07:15+00:00,2022-01-13 22:38:13+00:00,{8FE261E3-9548-4DA5-83C6-708C067FEDB4},0.0,None,None,0.012890,-8.157445e-06,"MULTIPOLYGON (((-101.30583 35.67490, -101.3058..."
9,14879,Bream Pond 2,Wildfire Final Fire Perimeter,Hand Sketch,0.026169,2022-08-05 20:50:01+00:00,2022-08-05 20:50:01+00:00,2022-08-05T20:50:00+00:00,0.026169,{B21AB665-88D2-43B0-8104-CDBFDB212418},...,None,2022-04-21 18:18:32+00:00,2022-01-07 13:35:31+00:00,{607B2BD8-7AEF-439D-8266-4D2A8C2D3D63},0.0,None,None,0.000431,-9.820952e-09,"MULTIPOLYGON (((-81.89454 29.16107, -81.89450 ..."
10,14880,Muck Farm 2,Wildfire Final Fire Perimeter,Phone/Tablet,0.121197,2022-08-05 21:14:02+00:00,2022-08-05 21:14:02+00:00,2022-08-05T21:14:02+00:00,0.121196,{4FAF10B7-E904-4497-94E9-80B2ABBA8F06},...,None,2022-02-26 21:50:07+00:00,2022-01-05 00:49:57+00:00,{54AC06A1-5890-40FC-8837-950D9ABE02F1},0.0,None,None,0.001109,-4.546450e-08,"MU

### Function to retrieve NDVI data from a single multipolygon

In [11]:
# function to retrieve NDVI data from a single multipolygon

def getNDVI(fire_polygon_json):
    
    featureCollection = ee.FeatureCollection(json.loads(fire_polygon_json)) #json.loads(fire_bounds_geom))

        # Any region of the world
    # polygon = ee.Geometry.Polygon([112.0, 1.0,
    #                               112.0, 1.5,
    #                               112.5, 1.5,
    #                               112.5, 1.0,
    #                               112.0, 1.0])
    startDate = '2021-01-01'
    endDate = '2023-04-01'

    # An ImageCollection is a stack or sequence of images
    modisNDVI = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').select('NDVI').filterDate(startDate, endDate)
        
    def getEarthEngineData(n):
        date = ee.Date(startDate).advance(n,'month')
        m = date.get("month")
        y = date.get("year")
        dic = ee.Dictionary({ # Create the earth engine dictionary
            'Date':date.format('yyyy-MM')
        })
        
        tempNDVI = (modisNDVI.filter(ee.Filter.calendarRange(y, y, 'year'))
                    .filter(ee.Filter.calendarRange(m, m, 'month'))
                    .mean()
                    .reduceRegion(
                        reducer = ee.Reducer.mean(),
                        geometry = featureCollection.geometry(),
                        scale = 250))
        return dic.combine(tempNDVI)

    modis_YrMo = ee.List.sequence(0, 12*2-1).map(getEarthEngineData)

    dataframe = gpd.GeoDataFrame(modis_YrMo.getInfo())

    return dataframe

### Function to pass a list of polygons to getNDVI() and save the results to a file

NOTE: Max rate of requests is 100/sec or 6000 requests/minute

In [28]:
# Function to pass a list of polygons to getNDVI() and save the results to a file

def collectAllNDVI(gdf):

    for i in range(len(gdf)): 
        
        # counters
        record = i
        next = record + 1

        # extract GeoDataFrame geometry to json
        fire_polygon_json = gdf.iloc[record:next].geometry.to_json()

        # extract the objectid
        objectid = gdf.iloc[record:next,0].values[0]

        # pass the json record for the fire to our getNDVI function 
        # to query Google Earth Engine for NDVI
        try: 
            
            start_time = timeit.default_timer()
            print(f'Getting NDVI data for objectid: {objectid}')

            # call NDVI function to retrieve NDVI as a pandas dataframe for a given polygon
            firedf = getNDVI(fire_polygon_json)
            elapsed = timeit.default_timer() - start_time
            print(f'Successfully retrieved NDVI data for objectid: {objectid} in {elapsed} seconds')
            
            # Add the OBJECTID to each record
            firedf['OBJECTID'] = objectid

            # save to file
            firedf.to_csv(f'ndvi data/{objectid}.csv', index=False)
            print(f'Saved NDVI data for objectid {objectid} at ndvi data/{objectid}.csv')
            # write confirmation to log file
            with open("output.out", 'a') as f:
                f.write(f'Saved NDVI data for objectid {objectid} at ndvi data/{objectid}.csv in {elapsed} seconds.\n')
                f.close()
        
        except Exception as error:
            print(f'Error: could not retrieve NDVI data for objectid: {objectid}')
            print(error)
            with open("output.out", 'a') as f:
                f.write(f'Error: could not save NDVI data for objectid {objectid}\n')
                f.close()


### Execute our functions to retrieve NDVI data for each wildfire

In [40]:
# define batch
batch = gdf.iloc[4500:5019]

# Collect all 
collectAllNDVI(batch)
print('Complete.')

Getting NDVI data for objectid: 20308
Successfully retrieved NDVI data for objectid: 20308 in 5.559259310000925 seconds
Saved NDVI data for objectid 20308 at ndvi data/20308.csv
Getting NDVI data for objectid: 20309
Successfully retrieved NDVI data for objectid: 20309 in 6.533100783002737 seconds
Saved NDVI data for objectid 20309 at ndvi data/20309.csv
Getting NDVI data for objectid: 20310
Successfully retrieved NDVI data for objectid: 20310 in 5.800145995999628 seconds
Saved NDVI data for objectid 20310 at ndvi data/20310.csv
Getting NDVI data for objectid: 20311
Successfully retrieved NDVI data for objectid: 20311 in 5.22668841000268 seconds
Saved NDVI data for objectid 20311 at ndvi data/20311.csv
Getting NDVI data for objectid: 20312
Successfully retrieved NDVI data for objectid: 20312 in 1.973678206995828 seconds
Saved NDVI data for objectid 20312 at ndvi data/20312.csv
Getting NDVI data for objectid: 20313
Successfully retrieved NDVI data for objectid: 20313 in 2.656253246001142

## Combine all the files into a single GeoPandas dataframe

In [106]:
import glob as glob
import pandas as pd
#define path to CSV files
path = r'ndvi data/*.csv'

#identify all CSV files
all_files = glob.glob(path)

#merge all CSV files into one DataFrame
fireNDVI = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
fireNDVI.head()

,Date,NDVI,OBJECTID
0,2021-01,-0.007771,19905
1,2021-02,0.010309,19905
2,2021-03,0.025273,19905
3,2021-04,0.102434,19905
4,2021-05,0.174388,19905


Check to see if we are missing any OBJECTIDs that need to be re-downloaded. These were downloaded in batches so there is the potential that some were missed.

In [72]:
# Get a list of missing values
missing_values = list(set(gdf['OBJECTID'].unique().tolist()) - set(fireNDVI['OBJECTID'].unique().tolist()))
# Subset gdf to isolate the missing values
missing_rows = gdf[gdf['OBJECTID'].isin(missing_values)]
print(f"There are {gdf['OBJECTID'].nunique() - fireNDVI['OBJECTID'].nunique()} missing OBJECTIDs")
print(f"These missing values are {missing_values}")

There are 0 missing OBJECTIDs
These missing values are []


Run collectNDVI for our missing rows

In [ ]:
# define batch
batch = missing_rows

# Collect all 
collectAllNDVI(batch)
print('Complete.')

## Analyze the NDVI data to find fires with greatest loss of vegetation

In [109]:
fireNDVI['Year'] = fireNDVI.Date.str.split(pat='-',expand=True)[0]
fireNDVI['Month'] = fireNDVI.Date.str.split(pat='-',expand=True)[1]
fireNDVI = fireNDVI[fireNDVI.Month.isin(['06','07','08'])]
fireNDVI

,Date,NDVI,OBJECTID,Year,Month
5,2021-06,0.292460,19905,2021,06
6,2021-07,0.236788,19905,2021,07
7,2021-08,0.245503,19905,2021,08
17,2022-06,0.466576,19905,2022,06
18,2022-07,0.269989,19905,2022,07
...,...,...,...,...,...
120438,2021-07,0.544583,20145,2021,07
120439,2021-08,0.476639,20145,2021,08
120449,2022-06,0.545477,20145,2022,06
120450,2022-07,0.547759,20145,2022,07


- find average of 3 months for 2021 for object id
- then find average of 3 months for 2022 for object id
- subtract 2022-2021
- low numbers indicate loss of vegetation
- find bottom five lowest

In [117]:
fireNDVI2021 = fireNDVI[fireNDVI.Year=='2021']
fireNDVI2021_mean = fireNDVI2021.groupby('OBJECTID')['NDVI'].agg('mean')
fireNDVI2021_mean

OBJECTID
14680    0.410065
14683    0.410572
14698    0.410572
14700    0.381710
14814    0.497753
           ...   
20866    0.405133
20867    0.386308
20868    0.299236
20869    0.606602
20870    0.437704
Name: NDVI, Length: 5019, dtype: float64

In [113]:
fireNDVI2022 = fireNDVI[fireNDVI.Year=='2022']
fireNDVI2022_mean = fireNDVI2022.groupby('OBJECTID')['NDVI'].agg('mean')
fireNDVI2022_mean

OBJECTID
14680    0.436276
14683    0.436169
14698    0.436169
14700    0.439309
14814    0.544982
           ...   
20866    0.417933
20867    0.417914
20868    0.386877
20869    0.467229
20870    0.554115
Name: NDVI, Length: 5019, dtype: float64

In [120]:
fire_mean_2021_2022 = pd.concat([fireNDVI2021_mean,fireNDVI2022_mean],axis=1)
fire_mean_2021_2022.columns.values[0] = 'NDVI 2021'
fire_mean_2021_2022.columns.values[1] = 'NDVI 2022'
fire_mean_2021_2022

,NDVI 2021,NDVI 2022
OBJECTID,,
14680,0.410065,0.436276
14683,0.410572,0.436169
14698,0.410572,0.436169
14700,0.381710,0.439309
14814,0.497753,0.544982
...,...,...
20866,0.405133,0.417933
20867,0.386308,0.417914
20868,0.299236,0.386877


In [127]:
fire_mean_2021_2022['difference'] = fire_mean_2021_2022['NDVI 2022'] - fire_mean_2021_2022['NDVI 2021']
fire_mean_2021_2022 = fire_mean_2021_2022.reset_index()
fire_mean_2021_2022.head()

,OBJECTID,NDVI 2021,NDVI 2022,difference
0,17199,0.393370,0.181567,-0.211803
1,16766,0.374368,0.163807,-0.210561
2,15664,0.482593,0.272709,-0.209884
3,17817,0.427967,0.218855,-0.209112
4,18220,0.420651,0.212321,-0.208330


Top 5 vegetation losses

In [131]:
ndvi_wfigs_fires = gdf.merge(fire_mean_2021_2022, on='OBJECTID')
ndvi_wfigs_fires.head()

,OBJECTID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_Acres_AutoCalc,poly_GlobalID,...,irwin_IsCpxChild,irwin_CpxName,irwin_CpxID,SHAPE_Length,SHAPE_Area,geometry,Month,NDVI 2021,NDVI 2022,difference
941,18778,Dead Horse,Wildfire Final Fire Perimeter,Auto-generated,0.116,2022-08-19 18:08:37+00:00,2022-08-19 18:08:37+00:00,2022-08-19T18:08:37+00:00,0.116086,{B159224E-E7EE-425B-825F-1DAE7C5FD641},...,0.0,None,None,0.001069,-5.146379e-08,"MULTIPOLYGON (((-120.74955 42.51717, -120.7495...",8,0.390712,0.201561,-0.189151
337,16570,Buck Lake,Wildfire Final Fire Perimeter,Auto-generated,0.100,2022-08-05 20:43:49+00:00,2022-08-05 20:43:49+00:00,2022-08-05T20:43:47+00:00,0.024378,{853FFC9E-D06E-4C56-B6B6-83F41AB683EA},...,0.0,None,None,0.000479,-1.176606e-08,"MULTIPOLYGON (((-94.56592 47.44970, -94.56589 ...",8,0.604398,0.418068,-0.186329
648,17670,Baker Draw,Wildfire Final Fire Perimeter,Mixed Methods,NaN,2022-07-09 21:45:50+00:00,2022-07-09 21:45:50+00:00,2022-07-09T21:45:49+00:00,34.731407,{77172620-1478-4D26-BFA2-ECB12643A3A0},...,0.0,None,None,0.024370,-1.497339e-05,"MULTIPOLYGON (((-104.50653 40.68982, -104.5065...",7,0.383551,0.218252,-0.165299
216,15988,CR 96 & CR 61,Wildfire Final Fire Perimeter,Hand Sketch,0.500,2022-08-05 21:11:50+00:00,2022-08-05 21:11:50+00:00,2022-08-05T21:11:50+00:00,2.463465,{A919A994-3E28-47DF-AC66-984512137CE1},...,0.0,None,None,0.005280,-1.061963e-06,"MULTIPOLYGON (((-104.50700 40.68258, -104.5069...",8,0.380399,0.222471,-0.157929
495,17127,Bay,Wildfire Final Fire Perimeter,Hand Sketch,0.100,2022-06-27 19:52:07+00:00,2022-06-27 19:52:07+00:00,2022-06-27T19:52:07+00:00,0.574305,{4ABAC022-E887-4349-83E9-F267B3C10C61},...,0.0,None,None,0.002016,-2.753877e-07,"MULTIPOLYGON (((-94.54200 47.10020, -94.54202 ...",6,0.607246,0.451525,-0.155721


In [138]:
fires_over_1000_acres = ndvi_wfigs_fires[ndvi_wfigs_fires['poly_GISAcres']> 1000]

fires_over_1000_acres = fires_over_1000_acres.sort_values(by='difference', ascending=True)
fires_over_1000_acres.head()

,OBJECTID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_Acres_AutoCalc,poly_GlobalID,...,irwin_IsCpxChild,irwin_CpxName,irwin_CpxID,SHAPE_Length,SHAPE_Area,geometry,Month,NDVI 2021,NDVI 2022,difference
305,16487,OVERFLOW,Wildfire Final Fire Perimeter,Mixed Methods,1893.401219,2022-06-15 20:30:35+00:00,2022-06-15 20:30:35+00:00,2022-06-15T20:30:35+00:00,1893.393384,{3CB31FA9-8217-4D4D-82B0-6B50A50E93EC},...,0.0,None,None,0.165997,-0.000742,"MULTIPOLYGON (((-104.35894 33.26867, -104.3593...",6,0.309907,0.203504,-0.106403
593,17453,Emigrant,Wildfire Final Fire Perimeter,Mixed Methods,1001.100000,2022-08-01 20:37:27+00:00,2022-08-01 20:37:27+00:00,2022-08-01T20:37:27+00:00,1001.140746,{8D15D42D-23CA-4C81-8327-7C297CBBBE63},...,0.0,None,None,0.103648,-0.000431,"MULTIPOLYGON (((-116.30027 40.65091, -116.2998...",8,0.264165,0.183010,-0.081155
375,16736,Buckthorn,Wildfire Final Fire Perimeter,Mixed Methods,1151.988000,2022-06-15 20:28:20+00:00,2022-06-15 20:28:20+00:00,2022-06-15T20:28:19+00:00,1151.983639,{1F3C5073-9454-4DA8-965C-E330746B7C7D},...,0.0,None,None,0.223092,-0.000445,"MULTIPOLYGON (((-103.80491 32.11353, -103.8048...",6,0.244795,0.185496,-0.059299
91,15544,BORDER,Wildfire Final Fire Perimeter,Hand Sketch,1774.236182,2022-08-05 21:09:14+00:00,2022-08-05 21:09:14+00:00,2022-08-05T21:09:12+00:00,1774.229617,{A2B39354-4446-43E2-8095-05821892591A},...,0.0,None,None,0.160702,-0.000681,"MULTIPOLYGON (((-111.28631 31.44798, -111.2864...",8,0.344187,0.301672,-0.042515
103,15604,Presumido Peak,Wildfire Final Fire Perimeter,Mixed Methods,2590.606814,2022-06-27 21:07:14+00:00,2022-06-27 21:07:14+00:00,2022-06-27T21:07:13+00:00,2590.597270,{4C496840-F6D8-4CD6-AD85-181186927E3C},...,0.0,None,None,0.197550,-0.000996,"MULTIPOLYGON (((-111.60621 31.57590, -111.6071...",6,0.356155,0.317788,-0.038367


In [139]:
fires_over_1000_acres[['OBJECTID','poly_IncidentName','poly_GISAcres','NDVI 2021','NDVI 2022','difference']]

,OBJECTID,poly_IncidentName,poly_GISAcres,NDVI 2021,NDVI 2022,difference
305,16487,OVERFLOW,1893.401219,0.309907,0.203504,-0.106403
593,17453,Emigrant,1001.100000,0.264165,0.183010,-0.081155
375,16736,Buckthorn,1151.988000,0.244795,0.185496,-0.059299
91,15544,BORDER,1774.236182,0.344187,0.301672,-0.042515
103,15604,Presumido Peak,2590.606814,0.356155,0.317788,-0.038367
412,16841,Elgin Bridge,2149.642448,0.274683,0.237317,-0.037366
475,17088,Contreras,29482.469648,0.314956,0.278186,-0.036770
317,16528,High Park,1572.201039,0.369352,0.333813,-0.035539
487,17113,Tonto Canyon,13562.480597,0.313586,0.295371,-0.018215
456,16993,Sawpit Creek,26243.145685,0.208861,0.192399,-0.016463
